# Prepare data for bert

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import itertools
import random

C:\Users\mirei\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\mirei\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\mirei\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)
C:\Users\mirei\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
# huntington's
nodes_hd = pd.read_csv('final data-kg/graph_nodes_v2024-05-27.csv') #04-02
edges_hd = pd.read_csv("final data-kg/graph_edges_v2024-05-27.csv") #04-02
pred_pairs = pd.read_csv("final data-kg/pairs_pred_iron_reduced.csv")
# pred_pairs_drugs = pd.read_csv("final data-kg/pairs_pred_drugs.csv")

C:\Users\mirei\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3417: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# alzheimer's
nodes_ad = pd.read_csv('final data-kg/alz_graph_nodes_v2024-05-25.csv')#"ALZHEIMER/monarch_nodes_v2024-05-25.csv")
edges_ad = pd.read_csv('final data-kg/alz_graph_edges_v2024-05-25.csv')#"ALZHEIMER/monarch_edges_v2024-05-25.csv")
# ad_dist_edges = pd.read_csv('final data-kg/ad_dist_edges.csv')
# nodes_ad.head()

### Prepare files

In [4]:
def triplet_file(nodes, edges, prop=True, def_prop = 'interacts_with'):
    '''
    Transforms data into the correct format for the model
    Input:
    nodes, edges: data
    prop: if there is a property associated with each edge
    def_prop: property to be set
    
    Output: dataframe to be saved as csv
    '''
    
    # get relevant info on entities/ prepare for merging
    subject = nodes.copy()
    object_ = nodes.copy()
    try:
        subject = subject.drop(columns=['semantic_groups'])
        subject = subject.rename(columns={"id": "subject_id", "preflabel": "subject_pl", "synonyms": "subject_syn", 
                                          "name": "subject_nm", "description": "subject_dsc"})

        object_ = object_.drop(columns=['semantic_groups'])
        object_ = object_.rename(columns={"id": "object_id", "preflabel": "object_pl", "synonyms": "object_syn", 
                                          "name": "object_nm", "description": "object_dsc"})
    except:
        pass
    
    # combine entities with edges
    merged1 = pd.merge(subject, edges, on='subject_id', how='right')
    merged2 = pd.merge(object_, merged1, on='object_id', how='right')
    print('merging worked:', len(merged1) == len(merged2))
    
    # fill nans with identifiers
    merged2['object_pl'] = merged2['object_pl'].fillna(merged2['object_id'])
    merged2['subject_pl'] = merged2['subject_pl'].fillna(merged2['subject_id'])
    
    if prop:
        merged2['property_label'] = merged2['property_label'].fillna(merged2['property_id'])

        # clean property label
        merged2['property_label'] = merged2['property_label'].str.replace('biolink:', '')
        merged2['property_label'] = merged2['property_label'].str.replace('_', ' ')
        
    else:
        # keep only the 3 columns
        merged2['property_label'] = def_prop
        
    # keep only the 3 columns
    triplets = merged2[['subject_pl', 'property_label', 'object_pl']]
    
    return triplets

In [ ]:
# ALZHEIMERS
triplets_ad = triplet_file(nodes_ad, edges_ad)
# saving
train, val = train_test_split(triplets_ad, test_size=0.2, random_state=42, shuffle=True)
train.to_csv('data-avui/train.csv', sep='\t', index=False)
val.to_csv('data-avui/val.csv', sep='\t', index=False)
print(len(train) + len(val) == len(edges_ad))

In [ ]:
# HUNTINGTON'S
triplets_hd = triplet_file(nodes_hd, edges_hd)
triplets_hd.to_csv('trip2.csv', index=None) 

In [5]:
# PAIRS
testing = triplet_file(nodes_hd, pred_pairs, prop=False)
testing.to_csv('data/test.csv', index=None) 
# testing.head()

merging worked: True


,subject_pl,property_label,object_pl
0,TFR2,interacts_with,MSR1
1,RFC5,interacts_with,PLOD2
2,NUBPL,interacts_with,GNL2
3,ADGRV1,interacts_with,SLC40A1
4,UQCRFS1,interacts_with,MAPRE1


In [ ]:
# DRUGS
testing_d = triplet_file(nodes_ad, pred_pairs_drugs, prop=False)
testing_d.to_csv('data/test_drugs.csv', index=None) 
# testing_d.head()

In [43]:
# ALZHEIMER'S + NEGATIVE
triplets_ad = triplet_file(nodes_ad, edges_ad)

df_neg = pd.read_csv("neg_p.csv")
neg = triplet_file(nodes_ad, df_neg, prop=False, def_prop='no interaction')
ad = pd.concat([triplets_ad,neg])
print(len(ad) == (len(triplets_ad)+len(neg)))

train, val = train_test_split(ad, test_size=0.2, random_state=42, shuffle=True)
train.to_csv('data/train_neg_tt.csv', sep='\t', index=False)
val.to_csv('data/val_neg_tt.csv', sep='\t', index=False)

merging worked: True
merging worked: True
True


---------

# split to load to neo4j

In [ ]:
print(len(edges_hd)/500000)
one = edges_hd[:500000] #500,000
two  = edges_hd[500000:1000000]
three = edges_hd[1000000:]
one.to_csv('data-neo4j/edges_hd_1.csv', index=False)
two.to_csv('data-neo4j/edges_hd_2.csv', index=False)
three.to_csv('data-neo4j/edges_hd_3.csv', index=False)

In [5]:
print(len(edges_ad)/250000)
one = edges_ad[:250000] #500,000
two  = edges_ad[250000:500000]
two_2  = edges_ad[500000:750000]
two_3  = edges_ad[750000:1000000]
three = edges_ad[1000000:1250000]
three_2 = edges_ad[1250000:]
one.to_csv('data-neo4j/edges_ad_1.csv', index=False)
two.to_csv('data-neo4j/edges_ad_2.csv', index=False)
two_2.to_csv('data-neo4j/edges_ad_3.csv', index=False)
two_3.to_csv('data-neo4j/edges_ad_4.csv', index=False)
three.to_csv('data-neo4j/edges_ad_5.csv', index=False)
three_2.to_csv('data-neo4j/edges_ad_6.csv', index=False)

5.802744
